In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
cd '/content/drive/My Drive/CSE599/AKI_COVID19/RRT_prediction'

/content/drive/My Drive/CSE599/AKI_COVID19/RRT_prediction


In [0]:
import pandas as pd
import numpy as np
dataframe = pd.read_csv("AKI_COVID_data_RRT_prediction.csv")

num_patients = 300
data = dataframe.loc[0:num_patients,:]


In [0]:
# SBP/DBP on admission. Have to split by '/' since they contain values in the form: 120/80
col_name = 'SBP/DBP on admission'
sbp = []
dbp = []
sbp.append(1) #since it is a number column
dbp.append(1) #since it is a number column
for i in range(num_patients):
  text = data[col_name][i+1].split('/')
  sbp.append(text[0])
  dbp.append(text[1])
  
data.insert(1, "SBP on admission", sbp, True) 
data.insert(1, "DBP on admission", dbp, True) 

In [0]:
#columns with dates not available in new sheet
# dates_col_list = ['Date of Admission', 'Date of Death/Discharge']

num_col_list = []
cat_col_list = []
target_col = ''
for column_name in data:
    if(data[column_name][0] == 1 or data[column_name][0] == '1'):
      num_col_list.append(column_name)
    elif(data[column_name][0] == 0 or data[column_name][0] == '0'):
      cat_col_list.append(column_name)
    elif(data[column_name][0] == 'Y'):
      target_col = column_name


In [0]:
data = data.loc[1:num_patients,:]


In [51]:
data.loc[1:5,:]

,Study ID,DBP on admission,SBP on admission,COVID19+ (y/n),Age,Sex,BMI,HTN (y/n),DM (y/n),CAD (y/n),Vascular disease (y/n),HF (y/n),CKD (y/n),CKD Stage,ESRD (y/n),COPD (y/n),Other lung diseases (y/n),Hx of ACEI Use (y/n),Hx of ARB Use(y/n),Status of ACE/ARB on admission,SBP/DBP on admission,MAP on admission,Cr at admmsion,Peak Cr during hospital stay,Cr at discharge,BUN at admission,Peak BUN,BUN/Cr at admission,BUN/Cr Peak,Ferritin on admission,Peak Ferritin Level,Change in Ferritin,LDH on admission,Peak LDH level,Change in LDH,ESR on admission,Peak ESR Level,Change in ESR,D-dimer on admission,Peak D-dimer level,Change in D-dimer,CRP on admission,Peak CRP level,Change in CRP,Admission Potassium,Admission eGFR,Length of Stay (days),AKI on admission? (y/n),AKI during hospitalization? (y/n),AKI Type,Recovery from AKI?,Hematuria? (y/n),Proetinuria? (y/n),UTI? (y/n),RRT (y/n),HD (y/n),CRRT/CVVHD (y/n),IVF use on admission,How much IVF (ml),What type of IVF,BNP on admission,Pulm edema on CXR on admission,ICU admission?,Mechanical ventilation (y/n),Patient Outcome (discharged vs death)
1,1.0,82,130,y,30,M,28.0,N,N,N,N,N,N,NaN,N,N,N,n,n,NaN,130/82,98,1.03,1.05,0.62,11,30,10.7,28.6,2141.0,2488.0,347.0,688.0,978.0,290.0,25.0,138.0,113.0,349.0,5251.0,4902.0,6.7,26.4,19.7,4.0,97,30.0,n,n,NaN,NaN,n,y,n,n,n,n,Y,1000,LR,143.0,N,Y,Y,Discharged
2,2.0,69,111,y,55,M,37.0,Y,Y,N,Y,N,N,NaN,N,N,N,n,n,NaN,111/69,83,1.06,1.22,0.82,11,28,10.4,23.0,NaN,1017.0,NaN,NaN,431.0,NaN,NaN,107.0,NaN,NaN,369.0,NaN,10.8,13.9,3.1,4.1,91,11.0,n,n,NaN,NaN,n,y,n,n,n,n,N,NaN,NaN,18.0,Y,N,N,discharged
3,3.0,77,111,y,53,F,NaN,N,N,N,N,N,N,NaN,N,N,N,n,n,NaN,111/77,88,0.96,0.96,0.67,9,12,9.4,12.5,527.2,527.2,0.0,333.0,333.0,0.0,59.0,59.0,0.0,331.0,331.0,0.0,15.4,15.4,0.0,4.9,78,5.0,n,n,NaN,NaN,NaN,NaN,NaN,n,n,n,Y,2000,LR,77.0,N,N,N,discharged
4,4.0,76,142,y,54,F,32.0,Y,N,N,N,N,N,NaN,N,N,N,n,n,NaN,142/76,98,0.87,1.12,0.87,14,19,16.1,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.3,88,9.0,n,n,NaN,NaN,y,n,n,n,n,n,Y,1000,LR,100.0,N,Y,N,discharged
5,5.0,70,125,y,39,M,27.0,N,N,N,N,N,N,NaN,N,N,N,n,n,NaN,125/70,88,1.46,1.46,1.15,7,13,4.8,8.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,69,5.0,y,n,1.0,y,n,n,n,n,n,n,Y,1000,LR,NaN,N,N,N,discharged


In [0]:
#One hot encoding 
def one_hot_encode(data, col_list):
    
    encoded_df = data[col_list[0]] #add dummy col, del in the end

    for i in range(len(col_list)):
        each_col_df = data[col_list[i]]
        each_col_df = each_col_df.astype(str).str.lower()
        each_col_df = pd.get_dummies(each_col_df, prefix=col_list[i] +"_is")
        encoded_df = pd.concat([encoded_df, each_col_df], axis=1)
        
    encoded_df = encoded_df.drop(columns=[col_list[0]])
    return encoded_df
    

In [0]:
encoded_df = one_hot_encode(data, cat_col_list)

#dropping is_nan columns after encoding
isnan_col_list = []
for column_name in encoded_df:
    if(column_name.endswith('is_nan')):
      isnan_col_list.append(column_name)

encoded_df = encoded_df.drop(isnan_col_list , axis=1)

In [0]:
numeric_df = data[num_col_list]

total_data = pd.concat([numeric_df, encoded_df], axis=1)

In [0]:

temp = data[target_col]
data[target_col] =  temp.astype(str).str.lower()
data[target_col]
i = data[((data[target_col] == 'y') | ( data[target_col] == 'n'))].index
My_list = [*range(1, num_patients+1, 1)] 
drop_indices = (list(set(My_list) - set(i)))

In [0]:
droppped_data = data.drop(drop_indices)
dropped_total_data = total_data.drop(drop_indices)


In [0]:

droppped_data[target_col] = pd.Categorical(droppped_data[target_col])
droppped_data['Y'] = droppped_data[target_col].cat.codes

In [0]:
def is_number_tryexcept(s):
    """ Returns True is string is a number. """
    try:
        float(s)
        return True
    except ValueError:
        return False

In [0]:
X = dropped_total_data.values

for i in range(X.shape[0]):
  for j in range(X.shape[1]):
    # if(pd.isnull(X[i][j])):
    #   X[i][j] = 0
    if(not pd.isnull(X[i][j])):
      if(not is_number_tryexcept(X[i][j])):
        X[i][j] = np.nan
      

In [0]:
Y = droppped_data['Y']

In [0]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
seed = 9
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [62]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [63]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 98.99%


In [64]:
y_score = model.predict_proba(X_test)[:,1]
from sklearn.metrics import average_precision_score, precision_recall_curve

curve_p, curve_r, thres = precision_recall_curve(y_test, y_score)
ap = average_precision_score(y_test, y_score)
ap

1.0

In [65]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int8)

In [66]:
np.asarray(y_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int8)